In [23]:
import pandas as pd
from datetime import datetime, timedelta

In [24]:
# 24:00을 시계열 자료형으로 변환하는 함수
def zerofrom24(datetime, _format):
    datetime = str(datetime)
    try:
        return pd.to_datetime(datetime, format = _format)
    except:
        datetime = datetime[:-5] + '00:00'
        return pd.to_datetime(datetime, format = _format)+ timedelta(days=1)

In [2]:
# 기상정보 가져오기(기온, 강수, 풍속 등)
df_temp_2018 = pd.read_csv('./data/기상관측/기상2018년.csv', encoding='cp949')
df_temp_2019 = pd.read_csv('./data/기상관측/기상2019년.csv', encoding='cp949')
df_temp_2020 = pd.read_csv('./data/기상관측/기상2020년.csv', encoding='cp949')
df_temp_2021 = pd.read_csv('./data/기상관측/기상2021년.csv', encoding='cp949')
df_temp_2022 = pd.read_csv('./data/기상관측/기상2022년.csv', encoding='cp949')

C:\Users\user\AppData\Local\Temp\ipykernel_2160\3079774618.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp_2018 = pd.read_csv('./data/기상관측/기상2018년.csv', encoding='cp949')
C:\Users\user\AppData\Local\Temp\ipykernel_2160\3079774618.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp_2019 = pd.read_csv('./data/기상관측/기상2019년.csv', encoding='cp949')
C:\Users\user\AppData\Local\Temp\ipykernel_2160\3079774618.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp_2020 = pd.read_csv('./data/기상관측/기상2020년.csv', encoding='cp949')
C:\Users\user\AppData\Local\Temp\ipykernel_2160\3079774618.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp_2021 = pd.read_csv('./data/기상관측/기상2021년.csv', encoding='cp949')
C:\Users\user\AppData\Local\Temp\ipykern

In [4]:
# 데이터 결합
df_temp = pd.concat([df_temp_2018, df_temp_2019, df_temp_2020, df_temp_2021, df_temp_2022], axis=0)

In [5]:
# 일시 컬럼을 시계열 자료형으로 변경
df_temp['일시'] = pd.to_datetime(df_temp['일시'], format='%Y%m%d %H:%M')

In [9]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4164450 entries, 0 to 832187
Data columns (total 15 columns):
 #   Column     Dtype         
---  ------     -----         
 0   지점         int64         
 1   지점명        object        
 2   일시         datetime64[ns]
 3   기온(°C)     float64       
 4   강수량(mm)    float64       
 5   풍속(m/s)    float64       
 6   풍향(16방위)   float64       
 7   습도(%)      float64       
 8   현지기압(hPa)  float64       
 9   해면기압(hPa)  float64       
 10  일조(hr)     float64       
 11  일사(MJ/m2)  float64       
 12  적설(cm)     float64       
 13  전운량(10분위)  float64       
 14  운형(운형약어)   object        
dtypes: datetime64[ns](1), float64(11), int64(1), object(2)
memory usage: 508.4+ MB


In [7]:
# 대기정보 가져오기(미세먼지, 초미세먼지, 이산화탄소, 일산화탄소, 아황산가스 농도)
df_airinfo = pd.read_csv('./data/관측소별대기정보/관측소2018~2022_수정.csv')

In [8]:
# 일시 컬럼을 시계열 자료형으로 변경
df_airinfo['일시'] = df_airinfo['일시'].apply(lambda x: zerofrom24(x, '%Y%m%d %H:%M'))

In [10]:
# 타입변경이 이루어진 데이터를 저장
df_airinfo.to_csv('./data/관측소별대기정보/관측소2018~2022_수정_타입변경.csv', index=False)

In [11]:
# 기상정보와 대기정보 데이터를 지점명과 일시를 기준으로 결합
df = pd.merge(df_temp, df_airinfo, how='left', left_on=['지점명', '일시'], right_on=['지점명', '일시'])

In [18]:
df.columns

Index(['지점', '지점명', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)',
       '현지기압(hPa)', '해면기압(hPa)', '일조(hr)', '일사(MJ/m2)', '적설(cm)', '전운량(10분위)',
       '운형(운형약어)', 'PM10', 'PM2.5', 'Ozon', 'CO2', 'CO', 'SO2'],
      dtype='object')

In [19]:
# 강수량과 적설 컬럼의 결측치를 0으로 채우기
df['강수량(mm)'].fillna(0, inplace=True)
df['적설(cm)'].fillna(0, inplace=True)

In [20]:
# 필요없는 컬럼 제거
df.drop(columns=['일사(MJ/m2)', '운형(운형약어)', '일조(hr)'], inplace=True)

In [26]:
# 결측값이 있는 행 제거
df.dropna(inplace=True)

In [27]:
df.to_csv('./data/기상대기정보.csv', index=False)